In [ ]:
#解压数据集 命令行运行要快些
# !tar -zxvf ./data/data49167/UCF-101-jpg.tgz
#拉取代码
# git clone https://gitee.com/thunder95/TPN #
# 选用的网络: r50f32s2.py




### 查看onnx网络结构
1. 修改原代码input的shape：
```
frames = frames.transpose((1, 0, 2, 3))
frames = np.expand_dims(frames, axis=0)
```

2. 里面改了些代码(注释了consensus)，cp算子也不好实现暂时屏蔽，通过netron可看到大概结构，onnx大于150mb无法上传
3. 也可以通过torch的summary查看结构和输出shape

### 构建网络

1. 没有设置nonlocal_cfg， 所以没有none local模块
2. cp算子的实现, 貌似check_point没有实际用途
3. inflate层的差异, 本网络直接输入相应的卷积大小，不加判断了
4. block总体来说就两种结构， 相互叠加
5. resnet50 3d, 输出out_indices最后两层
6. simple consensus, 没有写自定义算子，直接用的reduce_mean, 注意维度是否需要reshape新加一个维度seg_num
7. cls_head 没有平均池化层 fc层加了dropout=0.5

### 数据读取
1. 先测试UCF-101-jpg
2. 按照torch代码编写数据读取
3. 注意测试时num_segments=10, 需要修改相应的网络结果

In [1]:
#数据准备
# !tar -zxvf /home/aistudio/data/data49167/UCF-101-jpg.tgz -C /home/aistudio/work/
# !python work/TPN_paddle/avi2jpg.py
# !python work/TPN_paddle/ucf_jpg2pkl.py
# !python work/TPN_paddle/data_list_gender.py

#测试reader
%cd /home/aistudio/work/TPN_paddle/
from data_reader import KineticsReader
from cfg_config import parse_config, merge_configs, print_configs
config = parse_config('/home/aistudio/work/TPN_paddle/configs/tpn_config_1.txt')
# print(config)
train_config = merge_configs(config, 'test', {})
train_reader = KineticsReader('tsn', 'test', train_config ).create_reader()
# data, label = train_reader()
# print(data[0].shape)
for batch_id, data in enumerate(train_reader()):
    print("====>", data[0][0].shape)
    break

#测试test输出 (30, 96, 224, 224)
#测试train输出 (1, 96, 224, 224)

#数据维度变化 
#(batchsize，segnum，seglen×channel，224，224) #datareader输出
#（batchsize×segnum，seglen，3，224，224） #reshape
#（batchsize×segnum，3，seglen，224，224） #transpose

/home/aistudio/work/TPN_paddle
{'MODEL': {'name': 'tpn', 'format': 'pkl', 'num_classes': 101, 'seg_num': 1, 'seglen': 32, 'image_mean': [104, 117, 128], 'image_std': [1, 1, 1], 'num_layers': 50, 'new_length': 32, 'new_step': 2, 'modality': 'RGB'}, 'TRAIN': {'epoch': 45, 'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024, 'batch_size': 2, 'use_gpu': True, 'num_gpus': 1, 'filelist': '/home/aistudio/UCF-101-jpg/train.list', 'learning_rate': 0.003, 'learning_rate_decay': 0.1, 'l2_weight_decay': 0.0001, 'momentum': 0.9, 'total_videos': 80}, 'VALID': {'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024, 'batch_size': 2, 'filelist': '/home/aistudio/UCF-101-jpg/val.list'}, 'TEST': {'seg_num': 10, 'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024, 'batch_size': 1, 'filelist': '/home/aistudio/UCF-101-jpg/test.list'}, 'INFER': {'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024,

### 从torch中提取权重, 并转为paddle
注意inflate层，权重复制扩展到3维，并除以3

`!python work/TPN_paddle/load_torch_res50.py`

导出的权重位于 work/torch_restnet50.pdparams

### 加载权重并训练


In [ ]:
%cd /home/aistudio/work/TPN_paddle/
!python train.py --use_gpu True --config /home/aistudio/work/TPN_paddle/configs/tpn_config_1.txt --epoch 100 --pretrain True

### 测试模型效果

In [ ]:
!python test.py --use_gpu True --batch_size 1 --config /home/aistudio/work/ECO-paddle/configs/hmdb_config_5.txt --weights /home/aistudio/work/ECO-paddle/checkpoints_models/best_model.pdparams

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 